In [ ]:
from research.config import set_project_root

set_project_root(levels_up=3, print_root=True)

In [ ]:
import pandas as pd
import numpy as np
from research.datasets import CRSP
from finance_byu.summarize import summary

In [ ]:
df = CRSP().df.copy()
df.head()

In [ ]:
# TODO: add ibes data into dataset group. Download and put on drive
ibes = pd.read_csv("/home/josh-chambers/Downloads/08-ibes_eps_analyst.csv", parse_dates=["caldt"])
ibes.head(5)

In [ ]:
df["mdt"] = df["date"].values.astype("datetime64[M]")
df.head(5)

In [ ]:
ibes["mdt"] = ibes["caldt"].values.astype("datetime64[M]")
ibes.head(5)

In [ ]:
ibes = ibes.drop(columns=["caldt"])

df = df.merge(ibes, on=["cusip", "mdt"], how="left")
df.head(5)

**Create Dispersion Variable and Lagged Variables**

In [ ]:
df["disp"] = df["stdev"] / np.abs(df["meanest"])

df["displag"] = df.groupby("permno")["disp"].shift()
df["displag3"] = df.groupby("permno")["disp"].shift(3)
df["prclag"] = df.groupby("permno")["prc"].shift()

df.head()

**Task 1 and 2**

+ Form equal-weight portfolios based on lagged dispersion.<br><br>

+ Add a spread portfolio. Report summary statistics.<br><br>


**Working off a new copy of the data**

I'm going to work off sub-selection of the data I have the non-queried/selected data to go back to when I to the lagged three months dispersion based portfolios. 

In [ ]:
df = df.query("displag == displag and prclag >= 5 and meanest > -inf").reset_index(drop=True)
df.head()

In [ ]:
df["bins"] = df.groupby("date")["displag"].transform(pd.qcut, 5, labels=False)

ew = (
    df.groupby(["date", "bins"])["ret"]
    .mean()
    .unstack(level="bins")
    .rename("p{:.0f}".format, axis="columns")
    * 100
)
ew.head()

In [ ]:
ew["spr"] = ew["p0"] - ew["p4"]
summary(ew).loc[["count", "mean", "std", "tstat", "pval"],].round(3)

**Task 3**

+ Compute the average number of stocks in the portfolios.<br><br>

In [ ]:
df.groupby(["date", "bins"])["ret"].count().unstack(level="bins").mean()

**Task 4**

+ Form equal-weight portfolios based on three month lagged dispersion.<br><br>

In [ ]:
df = df.query("displag3 == displag3 and prclag >= 5 and meanest > -inf").reset_index(drop=True)

df["bins3"] = df.groupby("date")["displag3"].transform(pd.qcut, 5, labels=False)

ew = (
    df.groupby(["date", "bins3"])["ret"]
    .mean()
    .unstack(level="bins3")
    .rename("p{:.0f}".format, axis="columns")
    * 100
)

In [ ]:
ew["spr"] = ew["p0"] - ew["p4"]
summary(ew).loc[["count", "mean", "std", "tstat", "pval"],].round(3)